# Flash Attention with Docker for Local Development and Scaling

This document provides a guide on how to set up Flash Attention using Docker for local development and scaling. It includes instructions for building the Docker image, running the container, and using Flash Attention in your projects.

## Prerequisites
- Docker installed on your machine
- A compatible GPU (NVIDIA) 
- NVIDIA Docker runtime installed

## Sections
- [Building the Docker Image](#building-the-docker-image)
- [Using Flash Attention Locally](#using-flash-attention)
- [Scaling with GCP Cloud Computing](#scaling-with-gcp-cloud-computing)

## Building the Docker Image

To build the Docker image for Flash Attention, follow these steps.

1. Clone the repository:
   ```bash
   git clone https://github.com/gabenavarro/MLContainerLab.git
   cd MLContainerLab
   ```
2. Build the Docker image:
   ```bashll
   docker build -f ./assets/build/Dockerfile.flashattn.cu128py26cp312 -t flash-attention:128-26-312 .
   ```

   > Note: The following steps are to development within the container. My tutorials will be run inside the container, so you can skip them if you are not interested in development.

3. Run the Docker container detached with terminal access and GPUs connected:
   ```bash
   docker run -dt \
   --gpus all \
   -v $(pwd):/workspace \
   --name flash-attention \
   --env NVIDIA_VISIBLE_DEVICES=all \
   --env GOOGLE_APPLICATION_CREDENTIALS=/workspace/assets/secrets/gcp-key.json \
   flash-attention:128-26-312
   ```
   > Note: The `-v $(pwd):/workspace` option mounts the current directory to `/workspace` in the container, allowing you to access your files from within the container.
   > Note: The `--env` options set environment variables for GPU visibility and Google Cloud credentials.
   > Note: The `--gpus all` option allows the container to use all available GPUs.

4. Open the container in VSCode: 
   ```bash
   code --folder-uri vscode-remote://dev-container+flash-attention/workspace
   ```
   > Note: This command opens the current directory in VSCode, allowing you to edit files directly in the container.
   > Note: You may need to install the Remote - Containers extension in VSCode to use this feature.
   > Note: You may need to install the Python extension in VSCode to use this feature.
   > Note: You may need to install the Jupyter extension in VSCode to use this feature.